In [1]:
import pandas as pd
import numpy as np
import Levenshtein
import re
import unidecode 
import json

data = pd.read_excel('../data/Banco de Súmulas.xlsx')

In [2]:
# keep only single classes
classes = ['470', '49er', '49erFX', 'Nacra 17']
data = data[data['Classe Vela'].isin(classes)]

In [3]:
# treating strings

# unidecode
data['Nome Competidor'] = data['Nome Competidor'].apply(unidecode.unidecode)

# replace more than one space by one space
data['Nome Competidor'] = data['Nome Competidor'].str.replace(r'\s+', ' ')

# substituir [;] por [virgula espaço]
data['Nome Competidor'] = data['Nome Competidor'].str.replace(r';', r', ')

# substituir [ / ] por [virgula espaço]
data['Nome Competidor'] = data['Nome Competidor'].str.replace(r' / ', r', ')

for index, row in data.iterrows():
    if re.match(r'^([A-Z]{2,})', row['Nome Competidor']):
        # ERTHAL Carol LAMARCA Felipe
        data.at[index, "Nome Competidor"] = re.sub(r'([a-z]{2,}) ([A-Z]{2,})', r'\1, \2', data.at[index, "Nome Competidor"])
                
        # ERTHAL CarolLAMARCA Felipe
        data.at[index, "Nome Competidor"] = re.sub(r'([a-z]{2,})([A-Z]{2,})', r'\1, \2', data.at[index, "Nome Competidor"])

    else:
        # Carol ERTHALFelipe LAMARCA
        data.at[index, "Nome Competidor"] = re.sub(r'([A-Z])([A-Z][a-z])', r'\1, \2', data.at[index, "Nome Competidor"])

        # Carol ERTHAL Felipe LAMARCA
        data.at[index, "Nome Competidor"] = re.sub(r'([A-Z]) ([A-Z][a-z])', r'\1, \2', data.at[index, "Nome Competidor"])

        # Carol ErthalFelipe Lamarca / Erthal CarolLamarca Felipe
        data.at[index, "Nome Competidor"] = re.sub(r'([a-z])([A-Z][a-z])', r'\1, \2', data.at[index, "Nome Competidor"])

# upper case
data['Nome Competidor'] = data['Nome Competidor'].str.upper()

# get unique names
names = data['Nome Competidor'].unique()
len(names)

C:\Users\Carol Erthal\AppData\Local\Temp\ipykernel_20512\221886864.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  data['Nome Competidor'] = data['Nome Competidor'].str.replace(r'\s+', ' ')


1423

In [4]:
# read json to dict
with open('..\data\similar_names_pairs.json', 'r') as fp:
    similar_names_dict = json.load(fp)

# for each key, substitute the values by the key in the data
for key in similar_names_dict:
    for value in similar_names_dict[key]:
        data['Nome Competidor'] = np.where(data['Nome Competidor'] == value, key, data['Nome Competidor'])

In [5]:
names = data['Nome Competidor'].unique()
# reorder names in alphabetical order
names = np.sort(names)
# for name in names print name
for name in names:
    print(name)

ADAM GLOGOWSKI, MACIEJ KRUSIEC
AHMED AL HASANI, ABDULRAHMAN AL MASHARI
AHMED AL HASANI, VIRGIL AUBRIOT
AKIRA SAKAI, RUSSELL AYLSWORTH
AKIRA TAKAYANAGI, SHINGEN FURUYA
AKSELI KESKINEN, RONI PATTERSON
AKSELI KESKINEN, SINEM KURTBAY
ALBANE DUBOIS, LILI SEBESI
ALBERT TORRES, ELIAS ARETZ
ALBIN BOMAN, SIMON ALM
ALEC ANDERSON, CHRISTOPHER BROCKBANK
ALEKSANDRA MELZACKA, KINGA LOBODA
ALEKSANDRA MELZACKA, SANDRA JANKOWIAK
ALEXANDER BRORSSON, OSCAR ANDERSSON
ALEXANDER HEINZEMANN, CLIFTON KARTNER
ALEXANDER HEINZEMANN, EDWARD COLEMAN
ALEXANDER HEINZEMANN, JUSTIN BARNES
ALEXANDER KOSCHATE, REECE PILLINGER
ALEXANDR MOSKVICHEV, KIRILL ELCHANINOV
ALEXANDRA MALONEY, MOLLY MEECH
ALEXANDRA MALONEY, OLIVIA HOBBS
ALEXANDRA STALDER, SILVIA SPERI
ALEXANDRA TEN HOVE, MARIAH MILLEN
ALICA STUHLEMMER, PAUL KOHLHOFF
ALICAN KAYNAR, BESTE KAYNAKCI
ALICE MOSS, AMANDA LJUNGGREN
ALICE MOSS, EBBA BERNTSSON
ALICE MOSS, HANNA JOHNSSON
ALICIA FRAS SAMSON, ELENA BARRIO GARCIA
ALICIA FRAS, CLARISSE BAISSAC
ALLISON SURRETTE, 

In [6]:
# output data to csv
data.to_csv('..\data\Banco de Súmulas - Pair Classes.csv', index=False)